<a href="https://colab.research.google.com/github/Madushani-Weerasekara/D11_Bidirectional-LSTM_fake-news-classifier/blob/main/D11_Bidirectional_LSTM_fake_news_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Fake News Classifier using Bidirectional LSTM RNN

In [1]:
import pandas as pd

In [7]:
df = pd.read_csv('/content/fake_or_real_news.csv', engine='python', error_bad_lines=False)

<ipython-input-7-af2d7ed5b0b4>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  df = pd.read_csv('/content/fake_or_real_news.csv', engine='python', error_bad_lines=False)
Skipping line 1701: unexpected end of data


In [8]:
df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [10]:
df.shape

(1699, 4)

In [11]:
df.isnull().sum()

Unnamed: 0    0
title         0
text          0
label         0
dtype: int64

In [13]:
# Get Independent features
x = df.drop('label', axis=1)

In [14]:
# Dependent feature
y = df['label']

In [15]:
# Check wheather dataset is balance or not
y.value_counts()

FAKE    872
REAL    827
Name: label, dtype: int64

In [16]:
import tensorflow as tf

In [17]:
tf.__version__

'2.15.0'

In [21]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional

In [22]:
#Vocabulary size
voc_size = 5000

In [23]:
messages = x.copy()

In [25]:
messages.head()

,Unnamed: 0,title,text
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello..."
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T..."
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...


In [26]:
messages.shape

(1699, 3)

In [27]:
import nltk
import re
from nltk.corpus import stopwords

In [28]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [29]:
# If there are null values use this code fix missing index values
# messages.reset_index(inplace=True)

In [30]:
# Dataset preprocessing
from nltk.stem import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
  print(i)
  review = re.sub('[^a-zA_Z]', ' ', messages['title'][i])
  review = review.lower()
  review = review.split()

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
  review = ' '.join(review)
  corpus.append(review)



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [35]:
corpus

['ou mell illari ear',
 'atch xact oment aul yan ommit olit uicid rump alli',
 'erri go ari gestur sympathi',
 'erni support witter erupt anger e tri warn',
 'attl ew ork hy rimari atter',
 'ehran',
 'irl orrifi hat atch oyfriend e eft ace ime n',
 'ritain chindler i',
 'act check rump linton command chief forum',
 'ran reportedli make new push uranium concess nuclear talk',
 'ith three linton owa glimps fire elud illari linton campaign',
 'onald rump hockingli eak eleg ame omehow ot ven ors',
 'trong olar torm ech isk oday ew ct',
 'ay america repar orld ar',
 'rump take ruz lightli',
 'ow women lead differ',
 'hock ichel bama illari aught lamor ate ape romot',
 'illari linton roubl america otic hing idden ictur',
 'hat ran bill bama like',
 'chart explain everyth need know partisanship america',
 'slipperi slope rump propos ban uslim',
 'pisod ail eplor special guest andi',
 'illari linton ake ipartisan appeal taten sland',
 'ew enat major leader main goal scari',
 'nferno verpopul y